In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load first Excel file
data = pd.read_excel('train.xlsx')

# Split data into features and target variable
X = data['Entity Title']  # Selecting 'Entity Title' as the only feature
y = data['Category']

# Preprocessing
tfidf = TfidfVectorizer()
X_title = tfidf.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_title, y, test_size=0.2, random_state=42)

# Model training
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Model evaluation
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

# Convert X_test (csr_matrix) to a DataFrame
X_test_df = pd.DataFrame.sparse.from_spmatrix(X_test, columns=tfidf.get_feature_names_out())

# Reset the index of X_test_df for proper alignment
X_test_df.reset_index(drop=True, inplace=True)

# Create a DataFrame with predicted category and actual category for the test set
output_df = pd.DataFrame({'Entity Title': X.iloc[X_test_df.index].values,
                          'Predicted Category': predictions,
                          'Actual Category': y_test})

# Display the output DataFrame
print(output_df)


Accuracy: 0.7802359882005899
                                           Entity Title  \
3377   Integrity e-Learning (Anti-Bribery & Corruption)   
3284  ISO 37001:2016 Anti-Bribery Management Systems...   
2548  ISO 37001:2016 Anti-Bribery Management Systems...   
1233                         ESG AND INSURANCE (ONLINE)   
438   STRENGTHENING CONFIDENCE IN SUSTAINABILITY REP...   
...                                                 ...   
2481  Deloitte & MACT Event | Corporate Treasurer's ...   
2340                                 SEMINAR TANAH 2022   
2400  BASIC SCAFFOLDING COMPETENCY TRAINING FOR FRAM...   
2345               PENTADBIRAN PEJABAT UNTUK SETIAUSAHA   
2639                        RISK MANAGEMENT FUNDAMENTAL   

         Predicted Category        Actual Category  
3377             Functional             Functional  
3284             Functional             Functional  
2548             Functional             Functional  
1233                    HSE                    HSE

In [ ]:
# Assuming 'output_df' is already generated from previous steps

# Filter rows where predicted category is different from actual category
wrong_predictions = output_df[output_df['Predicted Category'] != output_df['Actual Category']]

# Save the wrongly predicted categories to a new Excel file
wrong_predictions.to_excel('randomforest_wrongpredictions.xlsx', index=False)


In [ ]:
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

# Assuming you have already trained your model and vectorizer
# Example training code
# ...

# Save the trained model and TF-IDF vectorizer to files
joblib.dump(model, 'noID_trained_model.pkl')  # Save the model
joblib.dump(tfidf, 'noID_tfidf_vectorizer.pkl')  # Save the TF-IDF vectorizer


['noID_tfidf_vectorizer.pkl']

In [ ]:
!ls

'Learning Category Masterlist - no ID.xlsx'   noID_trained_model.pkl
 noID_tfidf_vectorizer.pkl		      sample_data


In [ ]:
import joblib
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Load the downloaded model and TF-IDF vectorizer
loaded_model = joblib.load('noID_trained_model.pkl')  # Load the model
loaded_tfidf = joblib.load('noID_tfidf_vectorizer.pkl')  # Load the TF-IDF vectorizer


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

# Load the new Excel file
new_data = pd.read_excel('test.xlsx')

# Check the columns in the DataFrame
print(new_data.columns)

# Assuming 'Entity ID' and 'Entity Title' are the correct column names
X_new = new_data[['Entity Title']]  # Selecting 'Entity Title'

# Load the previously trained TF-IDF vectorizer and model
loaded_tfidf = joblib.load('noID_tfidf_vectorizer.pkl')  # Load the TF-IDF vectorizer
loaded_model = joblib.load('noID_trained_model.pkl')  # Load the model

# Transform 'Entity Title' using the loaded TF-IDF vectorizer
X_new_title = loaded_tfidf.transform(X_new['Entity Title'])

# Make predictions on the new data
new_predictions = loaded_model.predict(X_new_title)

# Create a DataFrame with title and predicted category for the new data
new_output_df = pd.DataFrame({'Entity Title': X_new['Entity Title'].values,
                              'Predicted Category': new_predictions})

# Display the output DataFrame for the new data
print(new_output_df)

# Save the predictions to an Excel file locally
new_output_df.to_excel('predicted_data.xlsx', index=False)


Index(['Entity Title'], dtype='object')
                                          Entity Title Predicted Category
0    LEAN SIX SIGMA GREEN BELT CERTIFICATION PROGRA...         Leadership
1            Retail Academy - Career Coaching Bootcamp         Leadership
2           Hazardous Chemical Management at Workplace                HSE
3                                         AGILE LEADER         Functional
4    KURSUS PENGENALAN PERTOLONGAN CEMAS, CPR DAN A...                HSE
..                                                 ...                ...
744                    Psychology of People Management         Functional
745             Modul 3 - Business & Commercial Acumen         Functional
746       PROJECT MANAGEMENT ESSENTIALS FOR EXECUTIVES         Functional
747               PROJECT MANAGEMENT : COST MANAGEMENT         Functional
748  PMI Agile Certified Practitioner (PMI-ACP) Â� ...         Functional

[749 rows x 2 columns]


In [ ]:
!ls

actualtest_randomforest_wrongpredictions.xlsx  noID_trained_model.pkl  sample_data  train.xlsx
noID_tfidf_vectorizer.pkl		       predicted_data.xlsx     test.xlsx


In [ ]:
# Assuming 'output_df' is already generated from previous steps
import pandas as pd
# Load validation results
validation_results = pd.read_excel('predicted_data.xlsx')

# Filter rows where predicted category is different from actual category
wrong_predictions = validation_results[validation_results['Predicted Category'] != validation_results['Actual Category']]

# Save the wrongly predicted categories to a new Excel file
wrong_predictions.to_excel('actualtest_randomforest_wrongpredictions.xlsx', index=False)


In [ ]:
!ls

actualtest_randomforest_wrongpredictions.xlsx  predicted_data.xlsx  sample_data


# SUmmary


predict with uncategorized: 311 out of 1225 wrong predictions

114 out of 749 wrong (15.22%)
0.78 accuracy
